# Phased Array of Half-wave Dipole Antennas 
By Jordan Harris-Toovy, Fall 2022 <br>

This Jupyter notebook, which as created for the final project of OIT's PHY330 course, briefly describes the operation of half-wave dipole antennas, and phased arrays composed of them, followed by a simulation of various configurations. <br>

## What is a half-wave dipole antenna?
As most of you (the students and instructors in this course) already know, a half-wave dipole antenna is a special case of a linear antenna where the two antenna conductors are axially aligned and each have a length of quarter of the wavelength of the frequency of interest (combined length is a half-wavelength). As the voltage across the antenna oscillates, the _charge_ in current (accelerating charges) _causes_ the radiation of electromagnetic waves.

![Dipole Antenna](https://upload.wikimedia.org/wikipedia/commons/f/fe/Dipole_antenna_standing_waves_animation_6_-_5fps.gif)
_Image source: Wikipedia - Dipole antenna_ https://en.wikipedia.org/wiki/Dipole_antenna#Half-wave_dipole <br>

This simple antenna, can be modeled as a oscillating dipole. In particular, the far-field electromagnetic waves can be accuratly approximated with the equation:

$$
\vec{E}_{(r,\theta,t)} = -\frac{\mu_0 c_0 I_0}{2 \pi r}
\frac{cos{(\frac{\pi}{2}}cos{\theta})}{sin{\theta}}
e^{i(\omega t - k_0 r)}\hat{\theta}
$$

Or if the absolute value of the fields are not important,

$$
\vec{E}_{(\vec{r},\theta,t)} = E_0 \frac{cos{(2\pi)}}{r}
{(\frac{t}{T}-\frac{r}{\lambda})}(\hat{x}cos{(\theta)}-\hat{y}sin{(\theta)})sin{(\theta)}
$$

With an arbitary $E_0$, which is the approach taken in this notebook. <br>

## What is a phased array?
Having an antenna is nice and all, but its not alway practical the have a wide beam pattern; what if you want more of the radiated energy in a particular direction (or directions)? Fot that, we can rely on the fact that waves obey superposition. If multiple antennas are aranged together, their emitted EM-waves will combine into a (usually) more complex shape. By changing the ralative position of the antennas, certain properties, such as a high effecive directivity of the resultant waves. A common way to achieve this is to place half-wave dipole antennas in a line (or grid), orthogonal to the direction of oscillation, with a quarter of the of wavelength of interest spacing between them. <br>
As you will see later in this notebook, the more antennas that are added to this structure, the smaller the angle that the output power is confined to. <br>

While arrays have enormous utility, they can be taken a step further; by changing the relative phase of each antenna, the radiated beam can be 'steered' to point in particular diretion. This is particularly useful with '2D' arrays where azimuth and elevation control can be achieved.

![Phased Array](https://upload.wikimedia.org/wikipedia/commons/4/4a/Phased_array_animation_with_arrow_10frames_371x400px_100ms.gif)
_Image source: Wikipedia - Phased array_ https://en.wikipedia.org/wiki/Phased_array <br>

This is called beamforming, and is used extensively in radio frequency (RF) engineering. For example, if you are using a computer or phone (also a computer) with Wi-Fi® 6 or newer, then the RF waves your communications are using are being directed at you by a phased array in a wireless access point! <br><br>

With all of that said, lets simulate a phased array:

In [1]:
### Code by Jordan Harris-Toovy, adapted from code by Dr. Jesse Kinder
### PHY330, OIT Fall term, 2022

## Import necessary libraries
import ipywidgets as ipw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm

# Set default Matplotlib settings
%matplotlib inline
plt.rcParams['image.cmap'] = 'jet'

In [2]:
## Set constants
const_c = 3.0e+8    #Speed of light. Units: m/s
freq = 6.0e+9       #Wave frequency. Units: Hz
sigma = 1e-8        #Floating point offset
E_trunk = 1.0       #Clipping factor for output waveform. Decrease if rogue vectors appear

# Calculate secondary constants
wavelen = const_c / freq
period = 1 / freq
shift = wavelen/4

# Setup time array
time_step = period / 50
time_vals = np.arange(0, period, time_step)
time_len = len(time_vals)

# Calculate coordinate constants
coord_extent = 4 * wavelen
coord_step = wavelen / 10

In [3]:
## Create all coordinate systems (main + alternates)

# Setup main vertical axis
coordH = np.arange(0, coord_extent, coord_step)

# Setup main and shifted horizontal axes
coord1 = np.arange(-coord_extent, coord_extent, coord_step)
coord2 = np.arange((-coord_extent + shift), (coord_extent + shift), coord_step)
coord3 = np.arange((-coord_extent - shift), (coord_extent - shift), coord_step)
coord4 = np.arange((-coord_extent + shift * 2), (coord_extent + shift * 2), coord_step)
coord5 = np.arange((-coord_extent - shift * 2), (coord_extent - shift * 2), coord_step)
coord6 = np.arange((-coord_extent + shift * 3), (coord_extent + shift * 3), coord_step)
coord7 = np.arange((-coord_extent - shift * 3), (coord_extent - shift * 3), coord_step)

# Make grids out of axis pairs
X1, Z1 = np.meshgrid(coord1, coordH)
X2, Z2 = np.meshgrid(coord2, coordH)
X3, Z3 = np.meshgrid(coord3, coordH)
X4, Z4 = np.meshgrid(coord4, coordH)
X5, Z5 = np.meshgrid(coord5, coordH)
X6, Z6 = np.meshgrid(coord6, coordH)
X7, Z7 = np.meshgrid(coord7, coordH)

# Make polar versions of the grids (radius)
R1 = np.sqrt(X1**2 + Z1**2 + sigma)
R2 = np.sqrt(X2**2 + Z2**2 + sigma)
R3 = np.sqrt(X3**2 + Z3**2 + sigma)
R4 = np.sqrt(X4**2 + Z4**2 + sigma)
R5 = np.sqrt(X5**2 + Z5**2 + sigma)
R6 = np.sqrt(X6**2 + Z6**2 + sigma)
R7 = np.sqrt(X7**2 + Z7**2 + sigma)

# Make polar versions of the grids (angle)
Th1 = np.arctan2(X1, Z1)
Th2 = np.arctan2(X2, Z2)
Th3 = np.arctan2(X3, Z3)
Th4 = np.arctan2(X4, Z4)
Th5 = np.arctan2(X5, Z5)
Th6 = np.arctan2(X6, Z6)
Th7 = np.arctan2(X7, Z7)

In [4]:
## Electric field calculation function
def plot_e_field(time=0, Dipoles=1, Phase=0):
    
    # Convert phase shift into a time shift
    time_shift = Phase / (360.0 * freq)
    
    # Select the number of diples to plot and find the field for each dipole
    if Dipoles == 1:
        E0 = 2;   # Initial field electric magnitude. Scales negatively per extra dipole pair
        E1 =  E0 * wavelen * np.cos(Th1) * np.cos(2*np.pi * ((time * time_step)/period - R1/wavelen)) / R1   # Far-field approximation
        Ex = (E1 * np.cos(Th1))   # Get the field on each axis (and combine each dipole field)
        Ez = -(E1 * np.sin(Th1))
        
    elif Dipoles == 3:
        E0 = 0.75;
        E1 =  E0 * wavelen * np.cos(Th1) * np.cos(2*np.pi * ((time * time_step)/period - R1/wavelen)) / R1
        # Extra equations for each dipole
        E2 =  E0 * wavelen * np.cos(Th2) * np.cos(2*np.pi * (((time * time_step + time_shift))/period - R2/wavelen)) / R2
        E3 =  E0 * wavelen * np.cos(Th3) * np.cos(2*np.pi * (((time * time_step - time_shift))/period - R3/wavelen)) / R3
        Ex = (E1 * np.cos(Th1)) + (E2 * np.cos(Th2)) + (E3 * np.cos(Th3))
        Ez = -(E1 * np.sin(Th1)) - (E2 * np.sin(Th2)) - (E3 * np.sin(Th3))
    
    elif Dipoles == 5:
    
        E0 = 0.45;
        E1 =  E0 * wavelen * np.cos(Th1) * np.cos(2*np.pi * ((time * time_step)/period - R1/wavelen)) / R1
        E2 =  E0 * wavelen * np.cos(Th2) * np.cos(2*np.pi * (((time * time_step + time_shift))/period - R2/wavelen)) / R2
        E3 =  E0 * wavelen * np.cos(Th3) * np.cos(2*np.pi * (((time * time_step - time_shift))/period - R3/wavelen)) / R3
        E4 =  E0 * wavelen * np.cos(Th4) * np.cos(2*np.pi * (((time * time_step + 2 * time_shift))/period - R4/wavelen)) / R4
        E5 =  E0 * wavelen * np.cos(Th5) * np.cos(2*np.pi * (((time * time_step - 2 * time_shift))/period - R5/wavelen)) / R5
        Ex = (E1 * np.cos(Th1)) + (E2 * np.cos(Th2)) + (E3 * np.cos(Th3)) + (E4 * np.cos(Th4)) + (E5 * np.cos(Th5))
        Ez = -(E1 * np.sin(Th1)) - (E2 * np.sin(Th2)) - (E3 * np.sin(Th3)) - (E4 * np.sin(Th4)) - (E5 * np.sin(Th5))
        
    elif Dipoles == 7:
        
        E0 = 0.375;
        E1 =  E0 * wavelen * np.cos(Th1) * np.cos(2*np.pi * ((time * time_step)/period - R1/wavelen)) / R1
        E2 =  E0 * wavelen * np.cos(Th2) * np.cos(2*np.pi * (((time * time_step + time_shift))/period - R2/wavelen)) / R2
        E3 =  E0 * wavelen * np.cos(Th3) * np.cos(2*np.pi * (((time * time_step - time_shift))/period - R3/wavelen)) / R3
        E4 =  E0 * wavelen * np.cos(Th4) * np.cos(2*np.pi * (((time * time_step + 2 * time_shift))/period - R4/wavelen)) / R4
        E5 =  E0 * wavelen * np.cos(Th5) * np.cos(2*np.pi * (((time * time_step - 2 * time_shift))/period - R5/wavelen)) / R5
        E6 =  E0 * wavelen * np.cos(Th6) * np.cos(2*np.pi * (((time * time_step + 3 * time_shift))/period - R6/wavelen)) / R6
        E7 =  E0 * wavelen * np.cos(Th7) * np.cos(2*np.pi * (((time * time_step - 3 * time_shift))/period - R7/wavelen)) / R7
        Ex = (E1 * np.cos(Th1)) + (E2 * np.cos(Th2)) + (E3 * np.cos(Th3)) + (E4 * np.cos(Th4)) + (E5 * np.cos(Th5))  + (E6 * np.cos(Th6)) + (E7 * np.cos(Th7))
        Ez = -(E1 * np.sin(Th1)) - (E2 * np.sin(Th2)) - (E3 * np.sin(Th3)) - (E4 * np.sin(Th4)) - (E5 * np.sin(Th5)) - (E6 * np.sin(Th6)) - (E7 * np.sin(Th7))
        
    # Get the magnitude of the electric field for graphing
    E = np.sqrt(Ex**2 + Ez**2)
    
    #Truncate large values 
    bad = E > E_trunk
    Ex[bad] = E_trunk * Ex[bad] / E[bad]
    Ez[bad] = E_trunk * Ez[bad] / E[bad]
    
    # Actually plot the frame now
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.contourf(X1, Z1, (np.sqrt(Ex**2 + Ez**2)))
    ax.set_xlabel('X')
    ax.set_ylabel('Z')
    plt.show()

# Use an interactive widget to get process parameters and animate the plot
ipw.interact(plot_e_field, time=ipw.Play(min=0, max=time_len), Dipoles=(1,7,2), Phase=(-75,75,5))

interactive(children=(Play(value=0, description='time', max=50), IntSlider(value=1, description='Dipoles', max…

<function __main__.plot_e_field(time=0, Dipoles=1, Phase=0)>

This plot is a looping animation of a phased array with a variable number of dipole elements and relative phase. It is interactive, so feel free to play with the sliders since the animation will automatically generate the correct frames for each input combination (though it might take a while to generate each frame).